This code seeks out to test how well deepseek v2-lite handles sentiment analysis in regards to israel-palestine conflict

Conda venv

In [1]:
!nvidia-smi

Sun Jun 29 13:01:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080      WDDM  |   00000000:65:00.0  On |                  N/A |
|  0%   34C    P8              8W /  225W |     778MiB /   8192MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
%pip uninstall torch torchvision torchaudio -y
%pip cache purge  # optional but can help clean corrupted wheels
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
%pip install safetensors

Found existing installation: torch 2.7.1+cu126
Uninstalling torch-2.7.1+cu126:
  Successfully uninstalled torch-2.7.1+cu126
Found existing installation: torchvision 0.22.1+cu126
Uninstalling torchvision-0.22.1+cu126:
  Successfully uninstalled torchvision-0.22.1+cu126
Found existing installation: torchaudio 2.7.1+cu126
Uninstalling torchaudio-2.7.1+cu126:
  Successfully uninstalled torchaudio-2.7.1+cu126
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Too many arguments


Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.7.1%2Bcu126-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.22.1%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.7.1%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.7.1%2Bcu126-cp311-cp311-win_amd64.whl (2717.0 MB)
Using cached https://download.pytorch.org/whl/cu126/torchvision-0.22.1%2Bcu126-cp311-cp311-win_amd64.whl (6.3 MB)
Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.7.1%2Bcu126-cp311-cp311-win_amd64.whl (4.2 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ------------

In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())  # Should return True

# Optional: Force a tensor on the GPU
x = torch.tensor([1.0, 2.0]).cuda()
print(x)

2.7.1+cu126
12.6
True
tensor([1., 2.], device='cuda:0')


In [3]:
%pip install transformers accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install safetensors

Note: you may need to restart the kernel to use updated packages.


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to your downloaded model
#model_path = "./DeepSeek-V2-Lite"
model_path = "./deepseek-llm-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map="auto",  # keeps only what's possible on GPU
    torch_dtype=torch.float16,
    offload_folder="./offload"
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.76s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [15]:
example_prompt = """Classify the political sentiment of each statement as either:
- pro-israel
- pro-palestine
- neutral

Example 1:
Text: The IDF is defending Israeli citizens.
Label: pro-israel

Example 2:
Text: The blockade has devastated Gaza.
Label: pro-palestine

Example 3:
Text: The situation is tragic for both sides.
Label: neutral
"""

# Clean label extractor
def extract_label(output):
    label = output.strip().lower().split()[-1]
    if "pro-is" in label:
        return "pro-israel"
    elif "pro-pal" in label:
        return "pro-palestine"
    elif "neutral" in label:
        return "neutral"
    else:
        return "unknown"

# Main function: one text at a time
def classify(text):
    prompt = example_prompt + f"\nNow classify this text:\nText: {text}\nLabel:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=3,
            do_sample=False,
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [16]:
pro_israel_texts = [
    "I stand with Israel in their right to defend themselves",
    "The IDF is protecting their citizens",
    "Israel has the right to exist as a Jewish state",
    "Israel has the right to defend its borders.",
    "Hamas is a terrorist organization targeting civilians.",
    "Supporting Israel is supporting democracy in the Middle East.",
    "The Iron Dome saves countless Israeli lives.",
    "Criticism of Israel often masks antisemitism.",
    "Israeli citizens live under constant rocket threat.",
    "The Jewish people have a historical right to this land.",
    "IDF operations aim to eliminate terrorist threats.",
    "Israel withdrew from Gaza, yet rockets still fly.",
    "The UN is biased against Israel in its resolutions."
]

# --- Pro-Palestine Texts ---
pro_palestine_texts = [
    "Free Palestine from occupation",
    "Palestinians deserve equal rights and freedom",
    "End the occupation of Palestinian territories",
    "Israel's blockade has devastated Gaza's economy.",
    "The occupation must end for peace to begin.",
    "Palestinian families are being evicted from their homes.",
    "The West Bank is under illegal military control.",
    "Free Gaza from siege and suffering.",
    "Palestinian children deserve safety and education.",
    "The wall separates families and stifles lives.",
    "The Nakba is an ongoing tragedy for Palestinians.",
    "Settlements violate international law.",
    "We must stand against apartheid policies."
]

# --- Neutral Texts ---
neutral_texts = [
    "This is a neutral comment about the situation",
    "The conflict is complex with valid arguments on both sides",
    "Both sides have suffered greatly in this conflict.",
    "Dialogue and understanding are essential for peace.",
    "The conflict has a long and complex history.",
    "Civilians on both sides deserve protection.",
    "International law should guide the resolution process.",
    "It's important to listen to all voices in this debate.",
    "War affects everyone, not just combatants.",
    "Social media often simplifies complex issues.",
    "Peace will require compromise from both parties.",
    "We must seek truth before taking sides."
]

sample_texts = pro_israel_texts + pro_palestine_texts + neutral_texts

results = []

for text in sample_texts:
    label = classify(text)
    decoded_label = extract_label(label)
    results.append(decoded_label)

    print(f"Text: {text}")
    print(f"Extracted Label: {decoded_label}")
    print("-" * 50)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Text: I stand with Israel in their right to defend themselves
Extracted Label: pro-israel
--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Text: The IDF is protecting their citizens
Extracted Label: pro-israel
--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...

Text: War affects everyone, not just combatants.
Extracted Label: neutral
--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Text: Social media often simplifies complex issues.
Extracted Label: neutral
--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Text: Peace will require compromise from both parties.
Extracted Label: neutral
--------------------------------------------------
Text: We must seek truth before taking sides.
Extracted Label: neutral
--------------------------------------------------


In [17]:
from sklearn.metrics import accuracy_score

true_labels = (
    ["pro-israel"] * len(pro_israel_texts)
    + ["pro-palestine"] * len(pro_palestine_texts)
    + ["neutral"] * len(neutral_texts)
)

# --- Accuracy ---
accuracy = accuracy_score(true_labels, results)
print(f"\nAccuracy: {accuracy * 100:.2f}%")


Accuracy: 94.74%


In [18]:
import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/reddit_comments_clean.csv')

label_mapping = {
    'with israel': 'pro-israel',
    'with palestine': 'pro-palestine',
    'neutral': 'neutral',
    'inquisitive': 'neutral',
    'indifferent': 'neutral'
}

df['sentiment'] = df['label'].map(label_mapping)
df = df.dropna(subset=['sentiment'])

true_labels = []
pred_labels = []

for _, row in df.iterrows():
    text = row['text']
    pred_id = classify(text)              # Predict (as label id)
    pred_str = extract_label(pred_id)     # Convert id to string
    pred_labels.append(pred_str)
    true_labels.append(row['sentiment'])

# Compute accuracy
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Test Accuracy: {accuracy:.4f}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

KeyboardInterrupt: 

In [23]:
# Compute accuracy
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5415
